## Create a Customer travel Destinations Map

In [22]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
#Import the WeatherPy_database.csv file 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,77.79,67,50,23.35,scattered clouds
1,1,Castro,BR,-24.7911,-50.0119,58.44,65,4,4.41,clear sky
2,2,Port Alfred,ZA,-33.5906,26.8910,61.50,33,100,10.25,overcast clouds
3,3,Fort Saint James,CA,54.4333,-124.2500,66.54,56,74,5.86,moderate rain
4,4,Palauig,PH,15.4350,119.9045,82.02,85,100,15.19,overcast clouds


In [6]:
# Ask the user to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [9]:
# Determine if there are empty rows
preferred_cities_df.count()

City_ID                266
City                   266
Country                265
Lat                    266
Lng                    266
Max Temp               266
Humidity               266
Cloudiness             266
Wind Speed             266
Current Description    266
dtype: int64

In [10]:
# get rid of rows with empty cells
preferred_cities_df.dropna(inplace = True)
preferred_cities_df.count()

/Users/ellenrafferty/.conda/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


City_ID                265
City                   265
Country                265
Lat                    265
Lng                    265
Max Temp               265
Humidity               265
Cloudiness             265
Wind Speed             265
Current Description    265
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,77.79,scattered clouds,-9.8000,-139.0333,
3,Fort Saint James,CA,66.54,moderate rain,54.4333,-124.2500,
5,Vila Do Maio,CV,77.79,scattered clouds,15.1333,-23.2167,
7,Saint-Philippe,RE,65.75,light rain,-21.3585,55.7679,
12,Bambous Virieux,MU,68.25,light intensity shower rain,-20.3428,57.7575,
19,Rikitea,PF,74.35,light rain,-23.1203,-134.9692,
26,Okhotsk,RU,67.80,clear sky,59.3833,143.3000,
27,Bengkulu,ID,73.62,clear sky,-3.8004,102.2655,
30,Bonfim,BR,77.16,heavy intensity rain,3.0833,-59.9500,
31,Pasighat,IN,76.55,overcast clouds,28.0667,95.3333,


In [ ]:
# Set parameters to search for hotels within 5000 meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [61]:
# get rid of empty cells
nan_value = float("NaN")

hotel_df_clean.replace("", nan_value, inplace=True)

hotel_df_clean.dropna(subset = ["Hotel Name"], inplace=True)



In [39]:
# Export the City_Data into a csv
output_data_file = "WeatherPy_vacation.csv"
hotel_df_clean.to_csv(output_data_file, index_label = "City_ID")

In [62]:
# Add city name, the country code, the weather description and maximum temperature for the city
# to info box template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} <span>&#176</span>F</dd>

</dl>
"""
#<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df_clean[["Lat", "Lng"]]

In [63]:
# Add marker layer for each city type to the map
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))